In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns 

import string 
import nltk
from nltk.corpus import stopwords 
from wordcloud import WordCloud 
nltk.download('stopwords')

import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau 

import warnings 
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/newemails/Ham.csv
/kaggle/input/newemails/Spam.csv
/kaggle/input/email-spam-classification-dataset-csv/emails.csv
/kaggle/input/emails/Ham_csv.numbers
/kaggle/input/emails/Spam_csv.numbers


# What are all these imports??
* seaborn - Python data visualization library based on Matplotlib
* string - provides string operations 
* nltk - Natural Language Toolkit library 
    * stopwords - list of common stopwords in multiple languages
* WordCloud - word cloud visualizations or visual representation of word data 
* pad sequences - pads sequences to the same length 

In [23]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix 

# Create a DataFrame 

In [24]:
ham = pd.read_csv('/kaggle/input/newemails/Ham.csv')

1 = spam, 0 = not spam

In [31]:
ham

,,,,,,,,Gmail export (1)
ID,Thread,Date,From,To,Subject,Snippet,Labels,Link
190502420645d278,190502420645d278,"Tue, 25 Jun 2024 16:04:15 -0000","""AnitaB.org"" <newsletter@anitab.org>",nh104@wellesley.edu,🎉 GHC 24 Registration Is Now OPEN!,"Join your community at GHC 24! View in Browser Step into your POWER - Join a powerful community of women and nonbinary technologists at GHC 24. | me + we | October 8-11, 2024 | Philadelphia, PA","CATEGORY_PROMOTIONS, UNREAD, IMPORTANT, INBOX",https://mail.google.com/mail/u/0/#inbox/190502...
1904f6f9899fe290,1904f6f9899fe290,"Tue, 25 Jun 2024 12:47:02 +0000","""Doug Dyer from @TheFundCFO Newsletter"" <thefundcfo@substack.com>",nh104@wellesley.edu,"#162 VCs Closing Q2 w/ Playbook, Model, Valuation Updates","👋 Hi, I&#39;m Doug! Welcome to @TheFundCFO crew! Every Tuesday/[Thursday], we publish VC/CFO insights that matter - highlights from notable VC GPs, LPs, and CFOs/finance pros. Check out our VC Fund","IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/1904f6...
1904f401a79c62cf,1904f401a79c62cf,"Tue, 25 Jun 2024 11:54:40 +0000","""Zvi Mowshowitz from Don't Worry About the Vase"" <thezvi@substack.com>",nh104@wellesley.edu,Monthly Roundup #19: June 2024,"Looks like we made it. Yes, the non-AI world still exists. Bad Governor New York Governor Kathy Hochul has gone rogue and betrayed New York City, also humanity, declaring a halt to congestion pricing a","IMPORTANT, CATEGORY_UPDATES, INBOX",https://mail.google.com/mail/u/0/#inbox/1904f4...
1904e3f0b57de850,1904e3f0b57de850,"Tue, 25 Jun 2024 07:14:32 +0000",Handshake <handshake@notifications.joinhandshake.com>,nh104@wellesley.edu,"2 new jobs match your saved search: ""Venture Capital""","Handshake 2 new jobs match your search for &quot;Venture Capital&quot; EisnerAmper logo EisnerAmper Accounting 2024 Portfolio Accountant Staff I $70–80K/yr Hybrid Job New York, NY Proskauer Rose LLP","UNREAD, IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/1904e3...
...,...,...,...,...,...,...,...,...
18fbba8376067a14,18fbba8376067a14,"Mon, 27 May 2024 16:05:00 -0400",Yutong Wu <yw108@wellesley.edu>,"CS Students <cs-students@wellesley.edu>, MAS-students <mas-students@wellesley.edu>, Neuroscience Club <neuroclub@wellesley.edu>, Cogsci-students@wellesley.edu, neuroscience-students@wellesley.edu, Data Science Community <data-science-community@wellesley.edu>",Recruiting Participants for MIT Media Lab Study🧠($35 Gift Card),"Hello Everyone! I hope you&#39;re making the most of the summer! If you&#39;re still around the Boston area, we have an exciting lab experiment at MIT Media Lab Fluid Interface Group that might","IMPORTANT, CATEGORY_FORUMS",https://mail.google.com/mail/u/0/#inbox/18fbba...
18fbaa690a5db832,18fbaa690a5db832,"Mon, 27 May 2024 17:22:09 +0200 (CET)","""AI for Good Global Summit, ITU"" <ai@itu.int>",Natalie Ho <nh104@wellesley.edu>,[AI for Good Summit] Happening this week! Important information for participants,"Important Information for Participants Dear colleagues, Thank you for registering to attend the AI for Good Global Summit in-person from 30 to 31 May 2024 in Geneva, Switzerland! The event is happening","UNREAD, IMPORTANT, CATEGORY_PERSONAL",https://mail.google.com/mail/u/0/#inbox/18fbaa...
18fb910214cc2312,18fb910214cc2312,"Mon, 27 May 2024 07:59:44 +0000",OxfordSpeaks <info@oxford-speaks.com>,<nh104@wellesley.edu>,Upcoming Events,"Brenda Hale and Gideon Levy View this email in your browser Upcoming Events Left: Brenda Hale. Right: Gideon Levy. Dear all, Welcome to Week 6! Here are the events we are hosting in the next few weeks:","IMPORTANT, CATEGORY_UPDATES",https://mail.google.com/mail/u/0/#inbox/18fb91...
18fb755840b95489,18fb755840b95489,"Sun, 26 May 2024 18:55:50 -0500",VAISU <info@vaisu.ai>,undisclosed-recipients:;,VAISU 2024: Many Thanks!,"Hi Everyone, We wanted to extend our heartfelt thanks to all participants and speakers for making the Virtual AI Safety Unconference 2024 a success. Your enga

In [33]:
ham["Subject"]

KeyError: 'Subject'

In [26]:
ham_essential = ham["Snippet"]
hm_essential

KeyError: 'Snippet'

# DataFrame 
* a dataframe is 2D data structure with columns 
* describe() is a method that returns the stats of a dataframe including its count, mean, standard deviation etc 

In [20]:
df.describe(include='object')

,Email No.
count,5172
unique,5172
top,Email 1
freq,1


In [9]:
df.shape

(115, 1)

In [24]:
df.columns()
df.target()

TypeError: 'Index' object is not callable

In [26]:
df['input'] = 'TEXT1: ' + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor 

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U16'), dtype('int64')) -> None

In [22]:
df.input.head()

0    0
1    0
2    0
3    0
4    0
Name: input, dtype: int64

# Tokenization 

Datasets is a library by HuggingFace that allows easily accessing and sharing datasets for Audio, Computer Vision, and NLP tasks. 

# Pandas Data Frame vs HF Dataset 
* Pandas DataFrame - 2D labeled data structure w columns of potentially different types 
* HF Dataset - library and data structure to store and access datasets 

In [ ]:
from datasets import Dataset, DatasetDict

#converts a pandas DataFrame into a Hugging Face Dataset object 
ds = Dataset.from_pandas(df)

In [ ]:
ds

This is a model developed by Microsoft called DeBERTa. It's the third version and has fewer parameters than the 'base' or 'large versions. 

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

# Data Preprocessing 

In [11]:
# Remove stopwords 
def remove_stopwords(text):
    stop_words = stopwords.words('english')
    star_words = []
    
    #store important words
    for word in str(text).split():
        word = word.lower()
        if word not in stop_words:
            star_words.append(word)
    output = " ".join(star_words)
    return output


In [30]:
ham['Subject'] = ham['Subject'].apply(lambda text: remove_stopwords(text))

KeyError: 'Subject'

# Transformers and AutoTokenizer 

* transformers - HF library with wide range of pre-trained models for NLP 
* AutoModelForSequenceClassification - loads appropriate model architecture 
* AutoTokenizer - loads appropriate tokenizer (processes text so that model can understand) 
* tokenizer splits the input text into tokens --> numbers --> adds any special tokens 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
tokz.tokenize("Hi my name is Natalie.")

In [ ]:
tokz('Hello my name is Nat')

In [ ]:
def tok_func(x): return tokz(x['input'])

Batch mapping allows you to speed up processing and freely control the size of the generated dataset. HF's Tokenizers library works faster with batches because it parallelizes the tokenization of all the examples in a batch. 

In [ ]:
tok_ds = ds.map(tok_func, batched=True)

In [ ]:
row = 